**Description**: Shows how to solve [Sudoku](http://en.wikipedia.org/wiki/Sudoku) puzzles using integer programming and JuMP.

**Author**: Iain Dunning

**License**: <a rel="license" href="http://creativecommons.org/licenses/by-sa/4.0/"><img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by-sa/4.0/88x31.png" /></a><br />This work is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by-sa/4.0/">Creative Commons Attribution-ShareAlike 4.0 International License</a>.

# Solving Sudoku with JuMP

<img style="width: 200px; height: auto" src="http://upload.wikimedia.org/wikipedia/commons/f/ff/Sudoku-by-L2G-20050714.svg">
<p style="text-align: center"><i>A partially solved Sudoku puzzle</i></p>

<a href="http://en.wikipedia.org/wiki/Sudoku">Sudoku</a> is a popular number puzzle. The goal is to place the digits 1,...,9 on a nine-by-nine grid, with some of the digits already filled in. Your solution must satisfy the following rules:

* The numbers 1 to 9 must appear in each 3x3 square
* The numbers 1 to 9 must appear in each row
* The numbers 1 to 9 must appear in each column

This isn't an optimization problem, its actually a *feasibility* problem: we wish to find a feasible solution that satsifies these rules. You can think of it as an optimization problem with an objective of 0.

We can model this problem using 0-1 integer programming: a problem where all the decision variables are binary. We'll use JuMP to create the model, and then we can solve it with any integer programming solver.

In [1]:
# Load JuMP
using JuMP

We will define a binary variable (a variable that is either 0 or 1) for each possible number in each possible cell. The meaning of each variable is as follows:

    x[i,j,k] = 1  if and only if cell (i,j) has number k

where `i` is the row and `j` is the column.

In [2]:
# Create a model
sudoku = Model()

# Create our variables
@defVar(sudoku, x[i=1:9, j=1:9, k=1:9], Bin)

9x9x9 Array{Variable,3}:
[:, :, 1] =
 x[1,1,1]  x[1,2,1]  x[1,3,1]  x[1,4,1]  …  x[1,7,1]  x[1,8,1]  x[1,9,1]
 x[2,1,1]  x[2,2,1]  x[2,3,1]  x[2,4,1]     x[2,7,1]  x[2,8,1]  x[2,9,1]
 x[3,1,1]  x[3,2,1]  x[3,3,1]  x[3,4,1]     x[3,7,1]  x[3,8,1]  x[3,9,1]
 x[4,1,1]  x[4,2,1]  x[4,3,1]  x[4,4,1]     x[4,7,1]  x[4,8,1]  x[4,9,1]
 x[5,1,1]  x[5,2,1]  x[5,3,1]  x[5,4,1]     x[5,7,1]  x[5,8,1]  x[5,9,1]
 x[6,1,1]  x[6,2,1]  x[6,3,1]  x[6,4,1]  …  x[6,7,1]  x[6,8,1]  x[6,9,1]
 x[7,1,1]  x[7,2,1]  x[7,3,1]  x[7,4,1]     x[7,7,1]  x[7,8,1]  x[7,9,1]
 x[8,1,1]  x[8,2,1]  x[8,3,1]  x[8,4,1]     x[8,7,1]  x[8,8,1]  x[8,9,1]
 x[9,1,1]  x[9,2,1]  x[9,3,1]  x[9,4,1]     x[9,7,1]  x[9,8,1]  x[9,9,1]

[:, :, 2] =
 x[1,1,2]  x[1,2,2]  x[1,3,2]  x[1,4,2]  …  x[1,7,2]  x[1,8,2]  x[1,9,2]
 x[2,1,2]  x[2,2,2]  x[2,3,2]  x[2,4,2]     x[2,7,2]  x[2,8,2]  x[2,9,2]
 x[3,1,2]  x[3,2,2]  x[3,3,2]  x[3,4,2]     x[3,7,2]  x[3,8,2]  x[3,9,2]
 x[4,1,2]  x[4,2,2]  x[4,3,2]  x[4,4,2]     x[4,7,2]  x[4,8,2]  x[4,9,2]
 

Now we can begin to add our constraints. We'll actually start with something obvious to us as humans, but what we need to enforce: that there can be only one number per cell.

In [3]:
for i = 1:9, j = 1:9  # Each row and each column
    # Sum across all the possible digits
    # One and only one of the digits can be in this cell, 
    # so the sum must be equal to one
    @addConstraint(sudoku, sum{x[i,j,k],k=1:9} == 1)
end

Next we'll add the constraints for the rows and the columns. These constraints are all very similar, so much so that we can actually add them at the same time.

In [4]:
for ind = 1:9  # Each row, OR each column
    for k = 1:9  # Each digit
        # Sum across columns (j) - row constraint
        @addConstraint(sudoku, sum{x[ind,j,k],j=1:9} == 1)
        # Sum across rows (i) - column constraint
        @addConstraint(sudoku, sum{x[i,ind,k],i=1:9} == 1)
    end
end

Finally, we have the to enforce the constraint that each digit appears once in each of the nine 3x3 sub-grids. Our strategy will be to index over the top-left corners of each 3x3 square with `for` loops, then sum over the squares.

In [5]:
for i = 1:3:7, j = 1:3:7, k = 1:9
    # i is the top left row, j is the top left column
    # We'll sum from i to i+2, e.g. i=4, r=4, 5, 6
    @addConstraint(sudoku, sum{x[r,c,k], r=i:i+2, c=j:j+2} == 1)
end

The final step is to add the initial solution as a set of constraints. We'll solve the problem that is in the picture at the start of the notebook. We'll put a `0` if there is no digit in that location.

In [6]:
# The given digits
init_sol = [ 5 3 0 0 7 0 0 0 0;
             6 0 0 1 9 5 0 0 0;
             0 9 8 0 0 0 0 6 0;
             8 0 0 0 6 0 0 0 3;
             4 0 0 8 0 3 0 0 1;
             7 0 0 0 2 0 0 0 6;
             0 6 0 0 0 0 2 8 0;
             0 0 0 4 1 9 0 0 5;
             0 0 0 0 8 0 0 7 9]
for i = 1:9, j = 1:9
    # If the space isn't empty
    if init_sol[i,j] != 0
        # Then the corresponding variable for that digit
        # and location must be 1
        @addConstraint(sudoku, x[i,j,init_sol[i,j]] == 1)
    end
end

In [7]:
# We are now ready to solve the problem
# For this to work, you must have an integer programming
# solver installed. If you don't, you can install one with
# Pkg.add("GLPKMathProgInterface")
# or
# Pkg.add("Cbc")
solve(sudoku)

:Optimal

To display the solution, we need to look for the values of `x[i,j,k]` that are 1.

In [8]:
# Extract the values of x
x_val = getValue(x)
# Create a matrix to store the solution
sol = zeros(Int,9,9)  # 9x9 matrix of integers
for i in 1:9, j in 1:9, k in 1:9
    # Integer programs are solved as a series of linear programs
    # so the values might not be precisely 0 and 1. We can just
    # round them to the nearest integer to make it easier
    if iround(x_val[i,j,k]) == 1
        sol[i,j] = k
    end
end
# Display the solution
println(sol)

[5 3 4 6 7 8 9 1 2
 6 7 2 1 9 5 3 4 8
 1 9 8 3 4 2 5 6 7
 8 5 9 7 6 1 4 2 3
 4 2 6 8 5 3 7 9 1
 7 1 3 9 2 4 8 5 6
 9 6 1 5 3 7 2 8 4
 2 8 7 4 1 9 6 3 5
 3 4 5 2 8 6 1 7 9]


Which is the correct solution:
<img style="width: 200px; height: auto" src="http://upload.wikimedia.org/wikipedia/commons/3/31/Sudoku-by-L2G-20050714_solution.svg">
<p style="text-align: center"><i>A completed Sudoku puzzle</i></p>